- [x] **Makes  `names` into  `molecules`**
- [x] **Makes  `molecules` into `atomic matrices`**

In [1]:
# Author : Manas Mahale <manas.mahale@bcp.edu.in>

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

%matplotlib inline

In [2]:
data = pd.read_csv("../Data/external/Dataset_1.tsv", sep='\t')
data = data.filter(['Ligand SMILES', 'IC50 (nM)'])
lig, ic50 = data['Ligand SMILES'], data['IC50 (nM)']

In [3]:
desc = pd.read_csv("../Data/processed/descriptors.csv").T[1:].T
col = desc.columns.to_list()

In [4]:
names = os.listdir("../Data/processed/")
names.remove("descriptors.csv")
names.remove(".DS_Store")
names.sort(key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])

### `a` is the numpy array [`(m, 3)`] holding `m`  `X, Y, Z` 


### `aa` is the numpy array [`(m, m)`] holding `m x m` atomic dostance matrix`

### `aaa` is the numpy array [`(number_of_molecules,)`] holding  `m x m atomic distance matrix` WITHOUT  `DIAGONAL MATRIX`

### `aaaa` is the numpy array `(number_of_molecules,molecular_descriptors)` holding  `m x m atomic` `distance matrix` WITH  `DIAGONAL MATRIX`

In [5]:
aaaa = []

for n, i in enumerate(names):
    with open(f"../Data/processed/{i}") as f:
        data = [i.split() for i in f.readlines()[10:]]

    a = pd.DataFrame(data)
    a.columns = ["ID", "ATOM", "X", "Y", "Z", "tmp"]
    atom = a["ID"].to_numpy()
    a = a.drop(['ATOM','ID','tmp'], axis = 1)
    a = a.to_numpy('float64')

    aa = []
    for j in a:
        Dist = []
        for k in a:
            Dist.append(np.linalg.norm(j - k))
        aa.append(Dist)
    aa = np.array(aa)
    
    aaa = []
    for l in col:
        np.fill_diagonal(aa, desc[l][n])
        aaa.append(aa)
    aaaa.append(np.array(aaa))
len(aaaa)

29

# Uncomment the next cell for Heatmaps

In [6]:
# for i in range(29):    
#     plt.figure(dpi=1000)
#     b = sns.heatmap(aaaa[i][0], cmap = 'tab20c',vmin = 0, vmax = 30)
#     b.axes.set_title(f"{i+1}'th molecule's Distance matrix populated with interatomic distances in the off-diagonal elements &\nwith {col[i]} in the diagonal",fontsize=7)
#     b.set_xlabel("Atomic Distances",fontsize=5)
#     b.set_ylabel("Atomic Distances",fontsize=5)
#     b.tick_params(labelsize=5)